# PYNQ DMA tutorial (Part 2: Using the DMA from PYNQ) 

---

## Aim 

* This notebook shows how to use the PYNQ DMA class to control an AXI DMA in a hardware design. This is the second part of a DMA tutorial. [PYNQ DMA tutorial (Part 1: Hardware design)](https://discuss.pynq.io/t/pynq-dma-tutorial-part-1-hardware-design/3133?u=cathalmccabe) shows how to build the Vivado hardware design used in this notebook. 


## References

* [PYNQ DMA tutorial (Part 1: Hardware design)](https://discuss.pynq.io/t/pynq-dma-tutorial-part-1-hardware-design/3133?u=cathalmccabe)
* [DMA Tutorial GitHub repository](https://github.com/cathalmccabe/pynq_tutorials/dma) with Tcl source to rebuild the hardware, and pre-compiled BIT and HWH
* [Xilinx PG021 AXI DMA product guide](https://www.xilinx.com/support/documentation/ip_documentation/axi_dma/v7_1/pg021_axi_dma.pdf) 


## Last revised
* 13 October 2021
   * Initial version


## Introduction

This overlay consists of an AXI DMA and an AXI Stream FIFO (input and output AXI stream interfaces). The FIFO connects the input and output streams of the DMA in a loopback configuration and will be used to explore the DMA and test the PYNQ DMA class.

![](./images/completed_design.png)

## Instantiate and download the overlay

In [ ]:
from pynq import Overlay

ol = Overlay("./dma_tutorial.bit")

We can check the IPs in this overlay using the IP dictionary (*ip_dict*).

In [ ]:
ol.ip_dict

Check help for the DMA object

In [ ]:
ol.axi_dma?

## Create DMA instances

Using the labels for the DMAs listed above, we can create two DMA objects.

In [ ]:
dma = ol.axi_dma
dma_send = ol.axi_dma.sendchannel
dma_recv = ol.axi_dma.recvchannel

## Read DMA
We will read some data from memory, and write to FIFO in the following cells.

The first step is to allocate the buffer. pynq.allocate will be used to allocate the buffer, and NumPy will be used to specify the type of the buffer. 

In [ ]:
from pynq import allocate
import numpy as np

data_size = 1000
input_buffer = allocate(shape=(data_size,), dtype=np.uint32)

The array can be used like any other NumPy array. We can write some test data to the array. Later the data will be transferred by the DMA to the FIFO. 

In [ ]:
for i in range(data_size):
    input_buffer[i] = i + 0xcafe0000

Let's check the contents of the array. The data in the following cell will be sent from PS (DDR memory) to PL (streaming FIFO).

### Print first few values of buffer 

In [ ]:
for i in range(10):
    print(hex(input_buffer[i]))

Now we are ready to carry out DMA transfer from a memory block in DDR to FIFO.

In [ ]:
dma_send.transfer(input_buffer)

## Write DMA
Let's read the data back from FIFO stream, and write to MM memory. The steps are similar.

We will prepare an empty array before reading data back from FIFO.

### Print first few values of buffer 
(Check buffer is empty)

In [ ]:
output_buffer = allocate(shape=(data_size,), dtype=np.uint32)

for i in range(10):
    print('0x' + format(output_buffer[i], '02x'))

In [ ]:
dma_recv.transfer(output_buffer)

The next cell will print out the data received from PL (streaming FIFO) to PS (DDR memory). This should be the same as the data we sent previously.

### Print first few values of buffer 

In [ ]:
for i in range(10):
    print('0x' + format(output_buffer[i], '02x'))

Verify that the arrays are equal

In [ ]:
print("Arrays are equal: {}".format(np.array_equal(input_buffer, output_buffer)))

## Check DMA status, and trigger an error
Check the error and idle status

In [ ]:
dma_recv.error?

In [ ]:
dma_recv.error

In [ ]:
dma_recv.idle?

In [ ]:
dma_recv.idle

First we will start a transfer, and check the DMA is not idle. We will then try to start another DMA transfer which shoudl trigger an error. 

In [ ]:
dma_recv.transfer(output_buffer)

In [ ]:
dma_recv.idle

Start another receive transfer while the DMA is not idle

In [ ]:
dma_recv.transfer(output_buffer)

We can check the *running* state of the DMA

In [ ]:
dma_recv.running?

In [ ]:
dma_recv.running

## Check the DMA register map
We can read back individual status bits as show above. It can be useful to read back the full register map which will give details on all control and status bits. The meaning of each register and each bit will not be covered. For more details you can refer to the product guide for the DMA.

In [ ]:
dma.register_map

As an example, we can compare the buffer (physical) addresses to the DMA source and destination addresses as shown in the register map. 

In [ ]:
print("Input buffer address   :", hex(input_buffer.physical_address))
print("Output buffer address  :", hex(output_buffer.physical_address))
print("---")
print("DMA Source address     :", hex(dma.register_map.MM2S_SA.Source_Address))
print("DMA Destination address:", hex(dma.register_map.S2MM_DA.Destination_Address))

## Free all the memory buffers
Don't forget to free the memory buffers to avoid memory leaks!

In [ ]:
del input_buffer, output_buffer